In [7]:
from sqlalchemy import func, between, and_
from Configuracion_BD import crear_conexion,obtener_sesion
import Tablas.Campus as Campus, Tablas.Carreras as Carreras, Tablas.Ciudades as Ciudades, Tablas.Municipios as Municipios, Tablas.Facultades as Facultades, Tablas.Paises as Paises, Tablas.Personas_carreras as Personas_carreras, Tablas.Personas as Personas, Tablas.Programas as Programas, Tablas.Provincias as Provincias, Tablas.Ramas as Ramas, Tablas.Tipo_persona as Tipo_persona, Tablas.Ubicaciones as Ubicaciones, Tablas.Genero as Genero

base_mysql="pil_trabajo_practico"  # nombre de la base de datos

engine_mysql = crear_conexion("mysql","root","Contadores2","127.0.0.1:4000",base_mysql) # Conexión a la base de datos

session_mysql = obtener_sesion(engine_mysql) # abro la sesión en la conexión establecida

#### Consulta cantidad de alumnos por carrera

In [ ]:
resultados=session_mysql.query(Programas.Programas.nombre,func.count(Personas_carreras.Personas_Carreras.carrera_id)).select_from(Personas.Personas).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).filter(Personas.Personas.tipo_id==2).group_by(Programas.Programas.nombre)
for x in resultados:
    print(f"En la carrera {x[0]} hay inscriptos {x[1]} estudiantes")

#### Consulta cantidad de profesores por carrera

In [ ]:
resultados=session_mysql.query(Programas.Programas.nombre,func.count(Personas_carreras.Personas_Carreras.carrera_id)).select_from(Personas.Personas).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).filter(Personas.Personas.tipo_id==1).group_by(Programas.Programas.nombre)
for x in resultados:
    print(f"En la carrera {x[0]} hay {x[1]} profesores")  

#### Consulta con el detalle de la cantidad de alumnos por provincia de origen y carrera que cursan

In [ ]:
resultados=session_mysql.query(Provincias.Provincias.nombre,Personas.Personas.nombre,Programas.Programas.nombre).select_from(Personas.Personas).join(Ubicaciones.Ubicaciones).join(Provincias.Provincias).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).group_by(Provincias.Provincias.nombre,Personas.Personas.nombre).order_by(Provincias.Provincias.nombre).filter(Personas.Personas.tipo_id==2)
for x in resultados:
    print(f"Provincia: {x[0]}, estudiante: {x[1]}, carrera que cursa: {x[2]}")

#### Consulta distribución de alumnos por carrera y grupo etario

##### Menores a 20 años

In [ ]:

resultados=session_mysql.query(Programas.Programas.nombre,Personas.Personas.nombre,Personas.Personas.age).select_from(Personas.Personas).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).group_by(Programas.Programas.nombre,Personas.Personas.nombre).where(and_(Personas.Personas.age<20,Personas.Personas.tipo_id==2))
for x in resultados:
    print(x)


##### Entre 20 y 30 años

In [ ]:
resultados=session_mysql.query(Programas.Programas.nombre,Personas.Personas.nombre,Personas.Personas.age).select_from(Personas.Personas).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).group_by(Programas.Programas.nombre,Personas.Personas.nombre).where(and_(Personas.Personas.age.between(20,30),Personas.Personas.tipo_id==2))
for x in resultados:
    print(x)

##### Mayores a 30 años

In [ ]:
resultados=session_mysql.query(Programas.Programas.nombre,Personas.Personas.nombre,Personas.Personas.age).select_from(Personas.Personas).join(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).group_by(Programas.Programas.nombre,Personas.Personas.nombre).where(and_(Personas.Personas.age>30,Personas.Personas.tipo_id==2))
for x in resultados:
    print(x)

#### Consulta listado de profesores relacionado con cada alumno

In [ ]:
profesores= session_mysql.query(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Personas.Personas).filter(Personas.Personas.tipo_id==1).order_by(Personas.Personas.nombre)
for profe in profesores:
    lista_alumnos=[]   
    alumnosporprofesor= session_mysql.query(Personas_carreras.Personas_Carreras).join(Carreras.Carreras).join(Programas.Programas).join(Personas.Personas).filter(and_(Personas.Personas.tipo_id==2,profe.carrera==Personas_carreras.Personas_Carreras.carrera))
    for alumno in alumnosporprofesor:        
        lista_alumnos.append(alumno.persona.nombre)
    print(f"El profesor {profe.persona.nombre} en la carrera {profe.carrera.programa.nombre} tiene a los siguientes alumnos: {lista_alumnos} \n")